# GPT-SoVITS v2 - Vietnamese Voice Cloning Pipeline (Final Fix v4)

Đây là notebook tối ưu hóa để huấn luyện (train) và sử dụng (inference) mô hình GPT-SoVITS v2 trên Google Colab.

### 🌟 Tính năng mới:
- **Fix lỗi WebUI ASR**: Thêm bước chạy ASR Tiếng Việt thủ công ngay trong Notebook (không cần làm trên WebUI).
- **Fix lỗi HFValidationError**: Tự động patch file config dùng đường dẫn tuyệt đối.
- **Tích hợp UVR5**: Tự động tách nhạc nền.

### 🚀 Quy trình thực hiện:
1. **Cài đặt môi trường**.
2. **Tải Dữ liệu & Làm sạch (UVR5)**.
3. **WebUI (Bước 1)**: Chỉ dùng để Cắt âm thanh (Slicing).
4. **ASR Thủ công**: Quay lại đây chạy Cell tạo phụ đề tiếng Việt.
5. **WebUI (Bước 2)**: Quay lại WebUI để Train và Test.

In [ ]:
# @title 1. Cài đặt Môi trường, UVR5 & GPT-SoVITS (Fix Download)
# @markdown Chạy cell này để cài đặt toàn bộ thư viện và tải model thủ công.

import os
import sys
from IPython.display import clear_output

# 1. Clone Repo GPT-SoVITS
if not os.path.exists("GPT-SoVITS-Vietnamese"):
    print("Dang clone repository GPT-SoVITS...")
    !git clone https://github.com/tqtuan8788-ai/GPT-SoVITS-Vietnamese.git
    %cd GPT-SoVITS-Vietnamese
else:
    %cd GPT-SoVITS-Vietnamese
    !git pull

# 2. Install Dependencies
print("Dang cai dat thu vien chinh...")
!pip install -q -r requirements.txt
!pip install -q openai-whisper

# 3. Cài đặt UVR5 (để tách nhạc/vocal)
print("Dang cai dat UVR5 va cac cong cu xu ly am thanh...")
!pip install -q soundfile librosa noise-reduction onnxruntime-gpu
# Tải model UVR5
!mkdir -p uvr5_weights
!wget -nc -P uvr5_weights https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/uvr5_weights/HP5_only_main_vocal.pth
!wget -nc -P uvr5_weights https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/uvr5_weights/onnx_dereverb_By_FoxJoy/vocals.onnx

# 4. Tải Pretrained Models THỦ CÔNG (Fix lỗi script download)
print("Dang tai pre-trained models thu cong tu Hugging Face...")

# Tạo đúng cấu trúc thư mục mà code yêu cầu: GPT_SoVITS/pretrained_models/...
base_model_dir = "GPT_SoVITS/pretrained_models"
os.makedirs(base_model_dir, exist_ok=True)
os.makedirs(f"{base_model_dir}/chinese-roberta-wwm-ext-large", exist_ok=True)
os.makedirs(f"{base_model_dir}/chinese-hubert-base", exist_ok=True)

# Tải Chinese-Roberta (Bắt buộc cho Text encoding)
!wget -nc -P {base_model_dir}/chinese-roberta-wwm-ext-large https://huggingface.co/hfl/chinese-roberta-wwm-ext-large/resolve/main/config.json
!wget -nc -P {base_model_dir}/chinese-roberta-wwm-ext-large https://huggingface.co/hfl/chinese-roberta-wwm-ext-large/resolve/main/pytorch_model.bin
!wget -nc -P {base_model_dir}/chinese-roberta-wwm-ext-large https://huggingface.co/hfl/chinese-roberta-wwm-ext-large/resolve/main/tokenizer.json

# Tải Chinese-Hubert (Bắt buộc cho Audio encoding)
!wget -nc -P {base_model_dir}/chinese-hubert-base https://huggingface.co/TencentGameMate/chinese-hubert-base/resolve/main/config.json
!wget -nc -P {base_model_dir}/chinese-hubert-base https://huggingface.co/TencentGameMate/chinese-hubert-base/resolve/main/pytorch_model.bin
!wget -nc -P {base_model_dir}/chinese-hubert-base https://huggingface.co/TencentGameMate/chinese-hubert-base/resolve/main/preprocessor_config.json

# Tải GPT-SoVITS v2 Pretrained Models (s2G, s2D, s1bert)
!wget -nc -P {base_model_dir} https://huggingface.co/lj1995/GPT-SoVITS/resolve/main/s2G488k.pth
!wget -nc -P {base_model_dir} https://huggingface.co/lj1995/GPT-SoVITS/resolve/main/s2D488k.pth
!wget -nc -P {base_model_dir} https://huggingface.co/lj1995/GPT-SoVITS/resolve/main/s1bert25Hz-2kh-longer-epoch=68e-step=50232.ckpt

# Tải G2PW Model (Cho tiếng Trung, nhưng script vẫn check)
!wget -nc -P GPT_SoVITS/text/G2PWModel https://paddlespeech.bj.bcebos.com/Parakeet/op_g2p_model/G2PWModel/G2PWModel_1.1.zip
!unzip -o -q GPT_SoVITS/text/G2PWModel/G2PWModel_1.1.zip -d GPT_SoVITS/text/G2PWModel

clear_output()
print("✅ Cài đặt hoàn tất! Đã tải đủ models thủ công.")

In [ ]:
# @title 3. Cắt Audio + Tạo Phụ Đề Tiếng Việt (ASR)
# @markdown Chạy cell này để cắt audio thành các đoạn ngắn và tạo file .list tiếng Việt.
# @markdown **Lưu ý:** Cell này sẽ chạy xong rồi dừng, không chạy liên tục.

import os
import subprocess

# Đảm bảo đang ở đúng thư mục repo
%cd /content/GPT-SoVITS-Vietnamese

# Cài đặt thêm dependencies cần thiết
!pip install -q faster-whisper

# Set PYTHONPATH để scripts tìm được modules
import sys
sys.path.insert(0, '/content/GPT-SoVITS-Vietnamese')
os.environ['PYTHONPATH'] = '/content/GPT-SoVITS-Vietnamese'

# ⚠️ Điền tên experiment của bạn
exp_name = "Giong_Doc_Sach_01" # @param {type:"string"}
input_audio_folder = f"/content/dataset/{exp_name}"
sliced_output_folder = "output/slicer_opt"
asr_output_file = f"output/{exp_name}.list"

os.makedirs(sliced_output_folder, exist_ok=True)
os.makedirs("output", exist_ok=True)

# ========== BƯỚC 1: CẮT AUDIO ==========
print("="*60)
print("🔪 BƯỚC 1: CẮT AUDIO THÀNH CÁC ĐOẠN NGẮN")
print("="*60)
print(f"📂 Input: {input_audio_folder}")
print(f"📂 Output: {sliced_output_folder}")

# Chạy slice audio với subprocess
slice_cmd = f'python tools/slice_audio.py "{input_audio_folder}" "{sliced_output_folder}" -34 4000 300 10 500 0.9 0.25 0 1'
print(f"Running: {slice_cmd}")
os.system(slice_cmd)

# Đếm file đã cắt
if os.path.exists(sliced_output_folder):
    sliced_files = [f for f in os.listdir(sliced_output_folder) if f.endswith((".wav", ".mp3", ".flac"))]
    print(f"\n✅ Đã cắt xong! Tổng cộng {len(sliced_files)} file audio.\n")
else:
    sliced_files = []
    print("\n⚠️ Thư mục output chưa có file.\n")

# ========== BƯỚC 2: ASR TIẾNG VIỆT ==========
print("="*60)
print("🎤 BƯỚC 2: NHẬN DẠNG GIỌNG NÓI (ASR) TIẾNG VIỆT")
print("="*60)
print(f"📂 Input: {sliced_output_folder}")
print(f"📄 Output: {asr_output_file}")
print("⏳ Đang chạy Whisper large-v3... Vui lòng chờ 2-10 phút.\n")

# Chạy ASR với subprocess - dùng os.system để biến được thay thế đúng
asr_cmd = f'python tools/asr/fasterwhisper_asr.py -i "{sliced_output_folder}" -o "{asr_output_file}" -s large-v3 -l vi -p float32'
print(f"Running: {asr_cmd}")
os.system(asr_cmd)

# Kiểm tra file .list đã tạo
if os.path.exists(asr_output_file):
    with open(asr_output_file, "r", encoding="utf-8") as f:
        lines = f.readlines()
    print(f"\n✅ ĐÃ TẠO FILE .LIST THÀNH CÔNG!")
    print(f"📄 Đường dẫn: {os.path.abspath(asr_output_file)}")
    print(f"📝 Tổng số dòng: {len(lines)}")
    print(f"\n📋 5 dòng đầu tiên:")
    for idx, line in enumerate(lines[:5]):
        print(f"  {idx+1}. {line.strip()[:80]}...")
else:
    print("❌ LỖI: Không tạo được file .list!")
    print(f"   Kiểm tra: Thư mục input có file audio không?")
    print(f"   Input folder: {input_audio_folder}")
    if os.path.exists(input_audio_folder):
        print(f"   Files: {os.listdir(input_audio_folder)[:5]}")
    else:
        print(f"   ⚠️ Thư mục input không tồn tại!")

print("\n" + "="*60)
print("✅ HOÀN TẤT! Tiếp tục chạy Cell 4 để mở WebUI.")
print("="*60)
print(f"\n👉 Khi vào WebUI, hãy điền đường dẫn này vào ô Text Label File:")
print(f"   {os.path.abspath(asr_output_file)}")


In [ ]:
# @title 4. Khởi động WebUI (Chạy sau cùng - Chạy liên tục)
# @markdown **QUAN TRỌNG:** Cell này sẽ chạy liên tục để duy trì WebUI.
# @markdown Bạn sẽ làm việc trên WebUI: Format data -> Train -> Inference

import os

# Đảm bảo đang ở đúng thư mục repo
%cd /content/GPT-SoVITS-Vietnamese

# Patch config.py
cwd = os.getcwd()
possible_bert_paths = [
    os.path.join(cwd, "GPT_SoVITS/pretrained_models/chinese-roberta-wwm-ext-large"),
    os.path.join(cwd, "pretrained_models/chinese-roberta-wwm-ext-large"),
    "/content/GPT-SoVITS/GPT_SoVITS/pretrained_models/chinese-roberta-wwm-ext-large"
]
found_bert = next((p for p in possible_bert_paths if os.path.exists(p)), possible_bert_paths[0])
found_hubert = found_bert.replace("chinese-roberta-wwm-ext-large", "chinese-hubert-base")

os.environ["cnhubert_base_path"] = found_hubert
os.environ["bert_path"] = found_bert
os.environ["colab_active"] = "1"
os.environ["is_share"] = "True"

print("🚀 Đang khởi động WebUI...")
print("⏳ Chờ khoảng 1-2 phút để WebUI load xong.")
print("\n" + "="*60)
print("📋 HƯỚNG DẪN SỬ DỤNG WEBUI:")
print("="*60)
print("1. Tab 1 -> 1A: Điền tên experiment và đường dẫn file .list")
print("2. Bấm nút Format data")
print("3. Tab 1 -> 1B: Bấm Train SoVITS, sau đó Train GPT")
print("4. Tab 1 -> 1C: Chọn model và tạo giọng nói")
print("="*60 + "\n")

!sed -i "s/is_share = False/is_share = True/g" config.py
!python webui.py --share --i18n_dir ./i18n/vi_VN.json

In [ ]:
# @title 4. [CHẠY SAU KHI SLICING] Tạo Phụ Đề Tiếng Việt (ASR)
# @markdown **Chạy cell này sau khi bạn đã bấm nút 'Audio Slicing' trên WebUI xong.**
# @markdown Nó sẽ tự động tìm các file đã cắt và tạo file .list tiếng Việt chuẩn.

import os

# Tên thư mục output (mặc định WebUI sẽ tạo ra folder này)
# Nếu bạn đổi tên experiment, hãy đổi cả ở đây
sliced_audio_folder = f"output/{exp_name}_sliced"
output_list_file = f"output/{exp_name}.list"

print(f"📂 Đang tìm file đã cắt tại: {sliced_audio_folder}")

if not os.path.exists(sliced_audio_folder):
    print("❌ LỖI: Không tìm thấy thư mục đã cắt! Bạn đã bấm nút 'Audio Slicing' trên WebUI chưa?")
else:
    print("🚀 Đang chạy Whisper (large-v3) để nhận dạng tiếng Việt...")
    # Chạy Whisper ASR
    !python tools/asr/faster_whisper_asr.py -i "{sliced_audio_folder}" -o "{output_list_file}" -s large-v3 -l vi -p float32
    
    print(f"\n✅ Đã xong! File kết quả nằm tại: {os.path.abspath(output_list_file)}")
    print("👉 Bước tiếp theo: Quay lại WebUI -> Tab '1B-Fine-tuned Training'.")
    print(f"👉 Copy đường dẫn này: '{os.path.abspath(output_list_file)}' dán vào ô 'Label file' rồi bấm 'Format data'.")

In [ ]:
# @title 5. Lưu Model về Drive
save_to_drive = True # @param {type:"boolean"}
drive_save_path = f"/content/drive/MyDrive/GPT_SoVITS_Models/{exp_name}"

if save_to_drive:
    from google.colab import drive
    if not os.path.exists('/content/drive'):
        drive.mount('/content/drive')
    
    os.makedirs(drive_save_path, exist_ok=True)
    print(f"Dang luu model {exp_name} vao Drive...")

    !cp GPT_SoVITS/pretrained_models/s2G*.pth "{drive_save_path}"/
    !cp GPT_SoVITS/pretrained_models/s1bert*.ckpt "{drive_save_path}"/
    !cp -r GPT_SoVITS/weights/* "{drive_save_path}"/ 2>/dev/null || true
    !cp -r logs/{exp_name}/* "{drive_save_path}"/ 2>/dev/null || true
    
    print(f"✅ Da luu xong tai: {drive_save_path}")